In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").appName("logistic-regression").getOrCreate()

23/03/26 21:51:44 WARN Utils: Your hostname, Moon-3.local resolves to a loopback address: 127.0.0.1; using 192.168.0.2 instead (on interface en0)
23/03/26 21:51:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/26 21:51:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
from pyspark.ml.linalg import Vectors 
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.feature import HashingTF, Tokenizer

In [5]:
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])
# spark라는 단어가 나올 때 마다 1이라는 label이 나옴 

In [6]:
# text라는 컬럼 token화 
tokenizer = Tokenizer(inputCol = "text", outputCol="words")
hashingTF = HashingTF(inputCol = tokenizer.getOutputCol(), outputCol = "features")

In [15]:
lr = LogisticRegression(maxIter=300, regParam=0.0001)
# instance 생성 

In [16]:
pipeline = Pipeline(stages = [tokenizer, hashingTF, lr])
# Pipeline 인스턴스 생성 
# 해당 파이프라인은 tokenizer -> hasingTF -> lr 순서로 데이터가 진행된다는 의미 

In [17]:
model = pipeline.fit(training)
# 학습 

In [18]:
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])

In [19]:
pred = model.transform(test)
pred.show()

+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
| id|              text|               words|            features|       rawPrediction|         probability|prediction|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  4|       spark i j k|    [spark, i, j, k]|(262144,[19036,68...|[0.70122328491106...|[0.66845893435750...|       0.0|
|  5|             l m n|           [l, m, n]|(262144,[1303,526...|[5.57896302429256...|[0.99623772853481...|       0.0|
|  6|spark hadoop spark|[spark, hadoop, s...|(262144,[173558,1...|[-2.5912529323975...|[0.06970349276517...|       1.0|
|  7|     apache hadoop|    [apache, hadoop]|(262144,[68303,19...|[7.16422654636539...|[0.99922682104138...|       0.0|
+---+------------------+--------------------+--------------------+--------------------+--------------------+----------+



In [20]:
pred.select(["id", "text", "probability", "prediction"]).show()

+---+------------------+--------------------+----------+
| id|              text|         probability|prediction|
+---+------------------+--------------------+----------+
|  4|       spark i j k|[0.66845893435750...|       0.0|
|  5|             l m n|[0.99623772853481...|       0.0|
|  6|spark hadoop spark|[0.06970349276517...|       1.0|
|  7|     apache hadoop|[0.99922682104138...|       0.0|
+---+------------------+--------------------+----------+

